# LIDC IDRI 2D SEGMENTATION WITH TERNARY CLASSES

## Import Libraries

In [1]:
import pandas as pd
import argparse
import os
from collections import OrderedDict
from glob import glob
import yaml
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torchvision import transforms
import torchsummary as summary

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, f1_score, jaccard_score, roc_curve
from tqdm import tqdm

from Unet_new.unet_model import UNet
from UnetNested.Nested_Unet import NestedUNet

In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

## Define Parameters

In [3]:
name = "UNet"           #default = "UNet"; can be NestedUNet
epochs = 100            #default = 400
batch_size = 12          #default = 12
early_stopping = 50     #default = 50
num_workers = 8        #default = 8
optimizer = 'Adam'      #default = 'Adam'; can be SGD
lr = 1e-5               #default = 1e-5
momentum = 0.9          #default = 0.9
weight_decay = 1e-4     #default = 1e-4
nesterov = False        #default = False
augmentation = True     #default = False

## Define Functions

### Dataset

In [4]:
class LidcDataset(Dataset):
    def __init__(self, IMAGES_PATHS, MASK_PATHS, transforms):
        self.image_paths = IMAGES_PATHS
        self.mask_paths = MASK_PATHS
        
        self.transforms = transforms

    def __getitem__(self, index):
        image = np.load(self.image_paths[index])
        mask = np.load(self.mask_paths[index])

        #Make image and mask 3 dimensional
        image = image.reshape(512,512,1)
        mask = mask.reshape(512,512,1)

        #Convert datatype
        mask = mask.astype('uint8')

        #Apply augmentation
        augmented = self.transforms(image=image,mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        mask = mask.reshape([1,512,512])

        image, mask = image.type(torch.FloatTensor), mask.type(torch.FloatTensor)     

        return image, mask
    
    def __len__(self):
        return len(self.image_paths)

In [5]:
transform = A.Compose([
            A.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
            A.HorizontalFlip(p=0.5),
            ToTensorV2()
        ])

### Metrics

In [6]:
def sensitivity_metric(target, output):
    mcm = multilabel_confusion_matrix(target, output, labels=[0,1,2,3])
    s0 = mcm[0,1,1]/(mcm[0,1,1]+mcm[0,1,0]) 
    s1 = mcm[1,1,1]/(mcm[1,1,1]+mcm[1,1,0])
    s2 = mcm[2,1,1]/(mcm[2,1,1]+mcm[2,1,0])
    s3 = mcm[3,1,1]/(mcm[3,1,1]+mcm[3,1,0])
    sensitivity = np.array([s0, s1, s2, s3])
    sensitivity[np.isnan(sensitivity)] = 0
    return sensitivity

### Utilities

In [7]:
def str_to_bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    #Computes and stores the average and current value
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Get Configuration

In [8]:
if augmentation == True:
    file_name = name + '_with_augmentation'
else:
    file_name = name + '_base'
os.makedirs('model_outputs/{}'.format(file_name), exist_ok=True)
print("Creating directory called ", file_name)

print('-' * 20)
print("Configuration Setting: ")
print("Model: ", name)
print("Max Epochs: ", epochs)
print("Batch Size: ", batch_size)
print("Number of Workers: ", num_workers)
print("Optimizer: ", optimizer)
print("Learning Rate: ", lr)
print("Augmentation: ", augmentation)

Creating directory called  UNet_with_augmentation
--------------------
Configuration Setting: 
Model:  UNet
Max Epochs:  100
Batch Size:  12
Number of Workers:  8
Optimizer:  Adam
Learning Rate:  1e-05
Augmentation:  True


## Create Model

In [9]:
criterion = torch.nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else torch.nn.CrossEntropyLoss()
cudnn.benchmark = True

#Creating the model
print("Creating model...")
if name == 'NestedUNet':
    model = NestedUNet(num_classes=4)
else:
    model = UNet(n_channels=1, n_classes=4)
model = model.cuda() if torch.cuda.is_available() else model

if torch.cuda.device_count() > 1:
    print("We can use ", torch.cuda.device_count(), " GPUs.")
    model = nn.DataParallel(model)

params = filter(lambda p: p.requires_grad, model.parameters())

if optimizer == 'Adam':
    optimizer = optim.Adam(params, lr=lr, weight_decay=weight_decay)
elif optimizer == 'SGD':
    optimizer = optim.SGD(params, lr=lr, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay)
else:
    raise NotImplementedError
    
summary.summary(model,(1,512,512))

Creating model...
Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64

Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64, 64]         --
|

## Load Dataset

In [10]:
#directory of Images and Masks folders (generated from preprocessing)                                         
IMAGE_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Image/'
MASK_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Mask/'                                                                 

#meta information
meta = pd.read_csv('/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Meta/meta_metastatic.csv')
# meta = meta[meta['patient_diagnosis'] != 0]

#Get train/test label from metadata file
meta['original_image'] = meta['original_image'].apply(lambda x: IMAGE_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")
meta['mask_image'] = meta['mask_image'].apply(lambda x: MASK_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")


#Split into training and validation
train_meta = meta[meta['data_split']=='Train']
val_meta = meta[meta['data_split']=='Validation']

#Get training images into list
train_image_paths = list(train_meta['original_image'])
train_mask_paths = list(train_meta['mask_image'])

#Get validation images into list
val_image_paths = list(val_meta['original_image'])
val_mask_paths = list(val_meta['mask_image'])

print("*"*50)
print("Original images: {}, masks: {} for training.".format(len(train_image_paths),len(train_mask_paths)))
print("Original images: {}, masks: {} for validation.".format(len(val_image_paths),len(val_mask_paths)))
print("Ratio between Validation and Training is {:2f}".format(len(val_image_paths)/len(train_image_paths)))
print("*"*50)


#Creating custom LIDC dataset
train_dataset = LidcDataset(train_image_paths, train_mask_paths, transforms=transform)
val_dataset = LidcDataset(val_image_paths, val_mask_paths, transforms=transform)

#Creating Dataloader
train_loader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  shuffle=True,
  pin_memory=True,
  drop_last=True,
  num_workers=num_workers
)
val_loader = DataLoader(
  val_dataset,
  batch_size=batch_size,
  shuffle=False,
  pin_memory=True,
  drop_last=False,
  num_workers=num_workers
)

**************************************************
Original images: 852, masks: 852 for training.
Original images: 353, masks: 353 for validation.
Ratio between Validation and Training is 0.414319
**************************************************


## Train the Model

In [11]:
torch.cuda.empty_cache()

In [12]:
log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','sensitivity','val_loss','val_iou', 'val_sensitivity'])
# log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','val_loss','val_iou'])

best_dice = 0
trigger = 0

for epoch in range(epochs):

    #Model Training
    avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter(), 'sensitivity': AverageMeter()}
#     avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter()}
    model.train()
    pbar = tqdm(total=len(train_loader)) #progress bar

    for i, data in enumerate(train_loader):

        input = data[0].cuda()
        target = data[1].cuda()
        output = model(input)

        #Get loss and metric
        target_metric = target.flatten().cpu()
        output_metric = torch.argmax(output, dim=1).flatten().cpu()
        loss = criterion(output, torch.argmax(target, dim=1))
        iou = jaccard_score(target_metric, output_metric, average=None)
        dice = f1_score(target_metric, output_metric, average=None)
        sensitivity = sensitivity_metric(target_metric, output_metric)

        #Calculate the gradient and perform optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Update average metrics
        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))
        avg_meters['dice'].update(dice, input.size(0))
        avg_meters['sensitivity'].update(sensitivity, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
            ('dice', avg_meters['dice'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    train_log = OrderedDict([
        ('loss', avg_meters['loss'].avg),
        ('iou', avg_meters['iou'].avg),
        ('dice', avg_meters['dice'].avg),
        ('sensitivity', avg_meters['sensitivity'].avg)
    ])


    #Model Validation
    val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter(), 'val_sensitivity': AverageMeter()}
#     val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter()}
    model.eval()

    with torch.no_grad():
        val_pbar = tqdm(total=len(val_loader))
        for i, val_data in enumerate(val_loader):

            val_input = val_data[0].cuda()
            val_target = val_data[1].cuda()
            val_output = model(val_input)
            
            val_target_metric = val_target.flatten().cpu()
            val_output_metric = torch.argmax(val_output, dim=1).flatten().cpu()
            val_loss = criterion(val_output, torch.argmax(val_target, dim=1))
            val_iou = jaccard_score(val_target_metric, val_output_metric, average=None)
            val_dice = f1_score(val_target_metric, val_output_metric, average=None)
            val_sensitivity = sensitivity_metric(val_target_metric, val_output_metric)

            val_avg_meters['val_loss'].update(val_loss.item(), val_input.size(0))
            val_avg_meters['val_iou'].update(val_iou, val_input.size(0))
            val_avg_meters['val_dice'].update(val_dice, val_input.size(0))
            val_avg_meters['val_sensitivity'].update(val_sensitivity, val_input.size(0))

            val_postfix = OrderedDict([
                ('val_loss', val_avg_meters['val_loss'].avg),
                ('val_iou', val_avg_meters['val_iou'].avg),
                ('val_dice', val_avg_meters['val_dice'].avg),
                ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
            ])
            val_pbar.set_postfix(val_postfix)
            val_pbar.update(1)
        val_pbar.close()

    val_log = OrderedDict([
        ('val_loss', val_avg_meters['val_loss'].avg),
        ('val_iou', val_avg_meters['val_iou'].avg),
        ('val_dice', val_avg_meters['val_dice'].avg),
        ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
    ])
    

    print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {},  Training IOU: {},  Training Sensitivity: {},  Validation Loss: {:.4f},  Validation DICE: {},  Validation IOU: {},  Validation Sensitivity: {}'.format(
        epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], train_log['sensitivity'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou'], val_log['val_sensitivity']
    ))
    # print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {:.4f},  Training IOU: {:.4f},  Validation Loss: {:.4f},  Validation DICE: {:.4f},  Validation IOU: {:.4f}'.format(
    #     epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou']
    # ))
#     print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {},  Training IOU: {},  Validation Loss: {:.4f},  Validation DICE: {},  Validation IOU: {}'.format(
#         epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou']
#     ))

    #Save values to csv file
    tmp = pd.Series([
        epoch,
        lr,
        train_log['loss'],
        train_log['iou'],
        train_log['dice'],
        train_log['sensitivity'],
        val_log['val_loss'],
        val_log['val_iou'],
        val_log['val_dice'],
        val_log['val_sensitivity']
    ], index=['epoch', 'lr', 'loss', 'iou', 'dice', 'sensitivity', 'val_loss', 'val_iou', 'val_dice', 'val_sensitivity'])
#     index=['epoch', 'lr', 'loss', 'iou', 'dice', 'val_loss', 'val_iou', 'val_dice'])
    

    log = log.append(tmp, ignore_index=True)
    log.to_csv('model_outputs/{}/log_metrics.csv'.format(file_name), index=False)

    trigger += 1

    #If best DICE score, save the model
    if np.mean(val_log['val_dice']) > best_dice:
        torch.save(model.state_dict(), 'model_outputs/{}/model_metrics.pth'.format(file_name))
        best_dice = np.mean(val_log['val_dice'])
        print("Saved new best model based on DICE metric!")
        trigger = 0
    
    if early_stopping >= 0 and trigger >= early_stopping:
        print("Early stopping.")
        break

    torch.cuda.empty_cache()

100%|██████████| 30/30 [01:11<00:00,  2.37s/it, val_loss=1.29, val_iou=[8.06901460e-01 3.04686112e-04 4.56371786e-03 2.70737206e-04], val_dice=[8.92592951e-01 6.08678638e-04 9.01257954e-03 5.40354328e-04], val_sensitivity=[0.8069331  0.30205891 0.12199099 0.02624523]]


Training Epoch 1/100,  Training Loss: 1.3429,  Training DICE: [7.49192046e-01 5.95509062e-04 4.81720042e-03 4.89800862e-04],  Training IOU: [6.35479035e-01 2.98095082e-04 2.41747073e-03 2.44990513e-04],  Training Sensitivity: [0.63553063 0.56728851 0.16418217 0.07208879],  Validation Loss: 1.2887,  Validation DICE: [8.92592951e-01 6.08678638e-04 9.01257954e-03 5.40354328e-04],  Validation IOU: [8.06901460e-01 3.04686112e-04 4.56371786e-03 2.70737206e-04],  Validation Sensitivity: [0.8069331  0.30205891 0.12199099 0.02624523]
Saved new best model based on DICE metric!


100%|██████████| 30/30 [01:10<00:00,  2.35s/it, val_loss=1.21, val_iou=[8.63227283e-01 3.10361818e-04 4.50297648e-03 3.44904259e-04], val_dice=[9.26298490e-01 6.19987128e-04 8.90438902e-03 6.87663295e-04], val_sensitivity=[0.86331846 0.2164417  0.13861515 0.01021944]]


Training Epoch 2/100,  Training Loss: 1.2375,  Training DICE: [0.91609033 0.00096075 0.00492489 0.00124781],  Training IOU: [8.45376747e-01 4.80898708e-04 2.47084774e-03 6.24614969e-04],  Training Sensitivity: [0.84545017 0.51416575 0.17594963 0.05339848],  Validation Loss: 1.2080,  Validation DICE: [9.26298490e-01 6.19987128e-04 8.90438902e-03 6.87663295e-04],  Validation IOU: [8.63227283e-01 3.10361818e-04 4.50297648e-03 3.44904259e-04],  Validation Sensitivity: [0.86331846 0.2164417  0.13861515 0.01021944]
Saved new best model based on DICE metric!


100%|██████████| 30/30 [01:07<00:00,  2.26s/it, val_loss=1.17, val_iou=[9.18219014e-01 1.55594556e-04 2.39798389e-03 3.17056376e-04], val_dice=[9.57234687e-01 3.11021641e-04 4.76269690e-03 6.31079140e-04], val_sensitivity=[0.91877725 0.08284917 0.04083006 0.00469628]]


...

Training Epoch 10/100,  Training Loss: 1.0926,  Training DICE: [0.99552297 0.         0.         0.        ],  Training IOU: [0.99108657 0.         0.         0.        ],  Training Sensitivity: [0.99184346 0.         0.         0.        ],  Validation Loss: 1.0911,  Validation DICE: [0.99593563 0.         0.         0.        ],  Validation IOU: [0.99191447 0.         0.         0.        ],  Validation Sensitivity: [0.99278333 0.         0.         0.        ]
Saved new best model based on DICE metric!


100%|██████████| 30/30 [01:06<00:00,  2.21s/it, val_loss=1.08, val_iou=[0.99431697 0.         0.         0.        ], val_dice=[0.99714778 0.         0.         0.        ], val_sensitivity=[0.9951882 0.        0.        0.       ]]    


Training Epoch 11/100,  Training Loss: 1.0857,  Training DICE: [9.96431170e-01 0.00000000e+00 0.00000000e+00 1.13928503e-04],  Training IOU: [9.92888414e-01 0.00000000e+00 0.00000000e+00 5.71269709e-05],  Training Sensitivity: [9.93646493e-01 0.00000000e+00 0.00000000e+00 9.72540815e-05],  Validation Loss: 1.0811,  Validation DICE: [0.99714778 0.         0.         0.        ],  Validation IOU: [0.99431697 0.         0.         0.        ],  Validation Sensitivity: [0.9951882 0.        0.        0.       ]
Saved new best model based on DICE metric!


 20%|██        | 6/30 [00:13<00:53,  2.25s/it, val_loss=1.08, val_iou=[0.99536318 0.         0.         0.        ], val_dice=[0.99767552 0.         0.         0.        ], val_sensitivity=[0.99592674 0.         0.         0.        ]]

ValueError: operands could not be broadcast together with shapes (4,) (3,) (4,) 